In [2]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import mnist

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
LATENT_DIMS = [64]
LATENT_DIM = 64
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\mnist_bigan_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "bald": bald_info_value_bigan,
}


In [5]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape[0], -1)

X_test = (X_test.astype(np.float32) - 127.5) / 127.5
X_test = X_test.reshape(X_test.shape[0], -1)


In [7]:
X_train_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\x_train_enhanced_bigan.npy")
X_test_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\x_test_enhanced_bigan.npy")

In [8]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [9]:
for latent_dim in LATENT_DIMS:
    for exp_iter in range(EXPERIMENT_COUNT):
        np.random.seed(exp_iter)
        initial_idx = np.array([],dtype=int)
        for i in range(10):
            idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
            initial_idx = np.concatenate((initial_idx, idx))

        for func_name, acquisition_func in ACQ_FUNCS.items():  

            X_initial = X_train_enhanced[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(latent_dim).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test_enhanced,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
            np.save(file_name, (acc_arr, dataset_size_arr))
        '''
        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(ORIGINAL_DIM).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join("results\\mnist_dbal", "{func_name}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter))
            np.save(file_name, (acc_arr, dataset_size_arr))
        '''

Accuracy after query 1: 0.4270
Accuracy after query 2: 0.4590
Accuracy after query 3: 0.5048
Accuracy after query 4: 0.4914
Accuracy after query 5: 0.5262
Accuracy after query 6: 0.5624
Accuracy after query 7: 0.5964
Accuracy after query 8: 0.6118
Accuracy after query 9: 0.6417
Accuracy after query 10: 0.6647
Accuracy after query 11: 0.6636
Accuracy after query 12: 0.6839
Accuracy after query 13: 0.6975
Accuracy after query 14: 0.7129
Accuracy after query 15: 0.7183
Accuracy after query 16: 0.7492
Accuracy after query 17: 0.7592
Accuracy after query 18: 0.7610
Accuracy after query 19: 0.7648
Accuracy after query 20: 0.7896
Accuracy after query 21: 0.7766
Accuracy after query 22: 0.7907
Accuracy after query 23: 0.7921
Accuracy after query 24: 0.8015
Accuracy after query 25: 0.8021
Accuracy after query 26: 0.7892
Accuracy after query 27: 0.8062
Accuracy after query 28: 0.8058
Accuracy after query 29: 0.8149
Accuracy after query 30: 0.8137
Accuracy after query 31: 0.8201
Accuracy after qu